In [4]:
import cv2
import numpy as np
from PIL import Image, ImageDraw
from IPython.display import display
from PIL.ExifTags import TAGS
import math
import time
from datetime import datetime

CROP THE ORIGINAL IMAGE INTO A SQUARE THAT IS TANGENT TO THE RECEIVER

In [62]:
def crop_square(image_path):
    img = Image.open(image_path)
    try:
        for orientation in TAGS.keys():
            if TAGS[orientation]=='Orientation':
                break
        exif=dict(img._getexif().items())
        if exif[orientation] == 3:
            img = img.transpose(Image.ROTATE_180)
        elif exif[orientation] == 6:
            img = img.transpose(Image.ROTATE_270)
        elif exif[orientation] == 8:
            img = img.transpose(Image.ROTATE_90)
    except (AttributeError, KeyError, IndexError):
        # cases where image does not have an orientation tag
        pass
    cropped_img = img.crop((190,555, img.width-170, img.height-690))
    cropped_image_path = image_path.replace('.jpg', '_cropped.jpg')
    cropped_img.save(cropped_image_path)
    return cropped_image_path

CALLS THE crop_square FUNCTION AND CROPS THE IMAGE CIRCULARLY OVER THE RECEIVER

In [63]:
def crop_circle(image_path):
    # Load the input image
    cropped_path = crop_square(image_path)
    img = cv2.imread(cropped_path)
    # Define the center and radius of the circular region to be cropped
    x = 180
    y = 175
    radius = (img.shape[1] - 10)//2

    # Create a mask of the same shape as the input image
    mask = np.zeros_like(img)

    # Draw a circle on the mask
    cv2.circle(mask, (x, y), radius, (255, 255, 255), -1)

    # Apply the mask to the input image
    cropped_img = cv2.bitwise_and(img, mask)

    # Save the cropped image
    cropped_image_path = cropped_path.replace('.jpg', '_circular.jpg')
    cv2.imwrite(cropped_image_path, cropped_img)

    return cropped_image_path



FUNCTION THAT GENERATES A MASK IN ORDER TO CALCULATE THE STANDARD DEVIATION WITHOUT TAKING THE BLACK PART INTO CONSIDERATION

In [64]:
def generate_mask(shape): # shape is the dimensions of the image
    # generate a grid of indices for the given shape
    x, y = np.indices(shape)
    # calculate the center of the circle
    center_x, center_y = shape[0] // 2, shape[1] // 2
    # calculate the radius of the circle
    radius = min(center_x, center_y) - 1
    # create a Boolean mask for values inside the circle
    mask = (x - center_x)**2 + (y - center_y)**2 <= radius**2
    return mask

In [65]:
generate_mask((355,360))

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

FUNCTION THAT CALCULATES THE STD OF A MASKED MATRIX

In [66]:
def calculate_std(matrix, mask):
    # convert matrix to numpy array
    matrix = np.array(matrix)
    # apply mask to matrix
    masked_matrix = np.ma.masked_array(matrix, mask=mask)
    # calculate the standard deviation along both axis
    std = np.std(masked_matrix, axis=None)
    return std

FUNCTION THAT PINPOINTS THE AVERAGE BRIGHTEST SPOT AND RETURNS ITS COORDINATES AND THE CENTER OF THE RECEIVER COORDINATES

In [67]:
def highlight_brightest_group(image_path):

    # Open image and convert it to grayscale
    img = Image.open(image_path).convert('L')
    img_arr = np.array(img)

    #Generate mask so that the black part of the photo isn't considered in the calculation of the standard deviation
    mask = generate_mask((355, 360))
    ecart = calculate_std(img_arr, mask)
    # ecart = np.std(img_arr, axis=None)
    print("Standard deviation: ", ecart)

    # Find the group of pixels with the highest brightness average
    brightest_group = np.argwhere(img_arr == img_arr.max())
    intensity = np.average(brightest_group)
    print("Intensity : ", intensity)
    brightest_x = brightest_group[:,1].mean()
    brightest_y = brightest_group[:,0].mean()

    # Add a red dot at the center of the brightest group and the center of the circle
    draw = ImageDraw.Draw(img)
    draw.ellipse((brightest_x-2, brightest_y-2, brightest_x+2, brightest_y+2), fill='red', outline='red')
    center_x = img.width//2
    center_y = img.height//2
    draw.ellipse((center_x-2, center_y-2, center_x+2, center_y+2), fill='red', outline='red')

    # Print the coordinates of the brightest group and the center of the circle
    print("Average brightest pixel coordinates: ({}, {})".format(int(brightest_x), int(brightest_y)))
    print("Circle center coordinates: ({}, {})".format(center_x, center_y))
    # img.show("Test")



    return center_x, center_y, int(brightest_x), int(brightest_y)

FUNCTION THAT CALCULATES THE ANGLE OF ROTATION NEEDED

In [68]:
def degree(img_url):
    l = highlight_brightest_group(img_url)
    x = l[0] - l[2]
    y = l[1] - l[3]
    opposite = y*0.15
    adjacent = 100  # Length of adjacent side
    angle_radians = math.atan2(opposite, adjacent)  # Calculate angle in radians
    angle_degrees = math.degrees(angle_radians)  # Convert radians to degrees
    print(f"The angle is {angle_radians:.2f} radians or {angle_degrees:.2f} degrees.")
    return angle_degrees

FUNCTION THAT COMPLETES THE WHOLE PROCESS (CROP, CIRCULAR CROP, BRIGHTEST POINT, ANGLE OF ROTATION)

In [69]:
def processImage(image_path):
    start_time = time.time()
    circular_crop_path = crop_circle(image_path)
    degree(circular_crop_path)
    end_time = time.time()
    print("Program run time : ", end_time-start_time, " seconds.")


In [70]:
processImage(r"receiver photos\omar.jpg")

Standard deviation:  10.224718628536001
Intensity :  194.26058441558442
Average brightest pixel coordinates: (194, 193)
Circle center coordinates: (180, 177)
The angle is -0.02 radians or -1.37 degrees.
Program run time :  0.050862789154052734  seconds.


In [71]:
img = Image.open(r"receiver photos\cloud1_circular.jpg").convert('L')
img_arr = np.array(img)
std = np.std(img_arr, axis=None)
print(std)

31.08179954514311


In [72]:
#runtime
#get_intensity

In [1]:
def JulianDate(year, month, day):
    if month <= 2:
        year -= 1
        month += 12
    A = year // 100
    B = 2 - A + A // 4
    JD_whole = int(365.25 * (year + 4716)) + int(30.6001 * (month + 1)) + day + B - 1524
    return JD_whole

In [2]:
JulianDate(2023,4,12)

2460047

In [5]:
def assignedate():
    now = datetime.now()
    year = now.year
    month = now.month
    zone = 2 #Lebanon is in the Eastern European Time Zone (EET) : UTC
    daylightsaving = 1
    day = now.day
    hour = now.hour - zone - daylightsaving
    minute = now.minute
    second = now.second
    print("Current Date & Time: ")
    print(year, '/', month, '/', day, ' (', now.strftime("%A"), ') ', now.strftime("%H:%M:%S"))

In [6]:
assignedate()

Current Date & Time: 
2023 / 4 / 12  ( Wednesday )  14:24:11


In [7]:
def sunpath():
    assignedate()    
    ###########
    now = datetime.utcnow()
    custom_format = now.strftime('%Y-%m-%d %H:%M:%S')
    hour = int(custom_format[11:13])
    minute = int(custom_format[14:16])
    second = int(custom_format[17:19])
    year = int(custom_format[:4])
    month = int(custom_format[5:7])
    day = int(custom_format[8:10])
    Lon = math.radians(35.59757830828219)
    Lat = math.radians(33.73835537379766)
    ###########
    , 
    T, JD_frac, L0, M, e, C, L_true, f, R, GrHrAngle, Obl, RA, Decl, HrAngle, elev, azimuth = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
    JD_whole = JulianDate(year, month, day)

    JD_frac = (hour + minute / 60. + second / 3600.) / 24. - .5
    T = JD_whole - 2451545
    T = (T + JD_frac) / 36525.
    L0 = math.radians(math.fmod(280.46645 + 36000.76983 * T,360)) #????
    M = math.radians(math.fmod(357.5291 + 35999.0503 * T,360))
    e = 0.016708617 - 0.000042037 * T
    C = math.radians(((1.9146 - 0.004847 * T) * math.sin(M)) + ((0.019993 - 0.000101 * T) * math.sin(2 * M)) + (0.00029 * math.sin(3 * M)))
    f = M + C
    Obl = math.radians(23 + 26 / 60. + 21.448 / 3600. - 46.815 / 3600 * T)
    JDx = JD_whole - 2451545
    GrHrAngle = 280.46061837 + (360 * JDx) % 360 + .98564736629 * JDx + 360.98564736629 * JD_frac
    GrHrAngle = math.fmod(GrHrAngle , 360.)
    L_true = math.fmod((C + L0), 2 * math.pi)
    R = 1.000001018 * (1 - e * e) / (1 + e * math.cos(f))
    RA = math.atan2(math.sin(L_true) * math.cos(Obl), math.cos(L_true))
    Decl = math.asin(math.sin(Obl) * math.sin(L_true))
    HrAngle = math.radians(GrHrAngle) + Lon - RA

    elev = math.asin(math.sin(Lat) * math.sin(Decl) + math.cos(Lat) * (math.cos(Decl) * math.cos(HrAngle))) #sun height
    # Azimuth measured eastward from north.
    azimuth = math.pi + math.atan2(math.sin(HrAngle), math.cos(HrAngle) * math.sin(Lat) - math.tan(Decl) * math.cos(Lat))

    print("{:.4f}".format(math.degrees(elev)), end=",")
    print("{:.4f}".format(math.degrees(azimuth)), end=",")
    print()
    global sunelevation
    sunelevation = elev
    print("sunelevation in RAD is equal to:", end="")
    print(sunelevation)
    global sunazimuth
    sunazimuth = azimuth / math.pi * 180.
    print("")
    print("sunazimuth in DEG is equal to:")
    print(sunazimuth)
    print("////////////////////////////////////")
    time.sleep(2)

In [8]:
sunpath()

Current Date & Time: 
2023 / 4 / 12  ( Wednesday )  14:24:35
55.0235,230.3615,
sunelevation in RAD is equal to:0.9603405758605691

sunazimuth in DEG is equal to:
230.3614992235293
////////////////////////////////////
